# Data Cleaning and Preparation

* During the course of doing data analysis and modeling, a significant amount of time
is spent on data preparation: loading, cleaning, transforming, and rearranging. Such
tasks are often reported to take up 80% or more of an analyst’s time. 
* Sometimes the
way that data is stored in files or databases is not in the right format for a particular
task. 
* Many researchers choose to do ad hoc processing of data from one form to
another using a general-purpose programming language, like Python, Perl, R, or Java,
or Unix text-processing tools like sed or awk. 
* Fortunately, pandas, along with the
built-in Python language features, provides you with a high-level, flexible, and fast set
of tools to enable you to manipulate data into the right form.


* In this chapter I discuss tools for missing data, duplicate data, string manipulation,
and some other analytical data transformations. 
* In the next chapter, I focus on combining and rearranging datasets in various ways.


In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 7.1  Handling Missing Data

* Missing data occurs commonly in many data analysis applications. 
* One of the goals
of pandas is to make working with missing data as painless as possible.
* For example,
all of the descriptive statistics on pandas objects exclude missing data by default.

* The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. 
* For numeric data, pandas uses the floating-point
value **NaN** (Not a Number) to represent missing data. 
* We call this a *sentinel* value that
can be easily detected:

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [3]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

* In pandas, we’ve adopted a convention used in the R programming language by referring to missing data as NA, which stands for *not available*. 
* In statistics applications,
NA data may either be data that does not exist or that exists but was not observed
(through problems with data collection, for example). 
* When cleaning up data for
analysis, it is often important to do analysis on the missing data itself to identify data
collection problems or potential biases in the data caused by missing data.

* The built-in Python **None** value is also treated as NA in object arrays:

In [5]:
string_data[0] = None

In [6]:
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [7]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

<img style="float: left;" src="pic/pic_7_1.png" width="600">

### Filtering Out Missing Data

* There are a few ways to filter out missing data. 
* While you always have the option to
do it by hand using **pandas.isnull** and boolean indexing, the **dropna** can be helpful.
* On a Series, it returns the Series with only the non-null data and index values:


In [8]:
from numpy import nan as NA

In [9]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [10]:
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [11]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is equivalent to:

In [12]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [13]:
data.notnull()

0     True
1    False
2     True
3    False
4     True
dtype: bool

* With DataFrame objects, things are a bit more complex. 
* You may want to drop rows
or columns that are all NA or only those containing any NAs. 
* **dropna** by default drops
any row containing a missing value:

In [14]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

In [15]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [16]:
cleaned = data.dropna()

In [17]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing **how='all'** will only drop rows that are all NA:

In [18]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass axis=1:

In [19]:
data[4] = NA

In [20]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [21]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data.   
Suppose
you want to keep only rows containing a certain number of observations.   
You can
indicate this with the **thresh** argument:

In [22]:
df = pd.DataFrame(np.random.randn(7, 3))

In [23]:
df

,0,1,2
0,-0.204708,0.478943,-0.519439
1,-0.555730,1.965781,1.393406
2,0.092908,0.281746,0.769023
3,1.246435,1.007189,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [24]:
df.iloc[:4, 1] = NA

In [25]:
df

,0,1,2
0,-0.204708,NaN,-0.519439
1,-0.555730,NaN,1.393406
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [26]:
df.iloc[:2, 2] = NA

In [27]:
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [28]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [29]:
df.dropna(thresh=2) # 2 개 이상의 non NA 값을 filtering out  한다.

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways.  
For most purposes, the **fillna** method is the workhorse function to use.   
Calling **fillna** with a constant replaces missing values with that value:

In [30]:
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [31]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


Calling **fillna** with a dict, you can use a different fill value for each column:

In [32]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [33]:
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


**fillna** returns a new object, but you can modify the existing object in-place:

In [34]:
df.fillna(0, inplace=True)

In [35]:
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


The same interpolation methods available for reindexing can be used with **fillna**:

In [36]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [37]:
df.fillna(method='ffill')

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [38]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


With **fillna** you can do lots of other things with a little creativity.   
For example, you might pass the mean or median value of a Series:


In [39]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [40]:
data.mean()

3.8333333333333335

In [41]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

<img style="float: left;" src="pic/pic_7_2.png" width="500">

## Data Transformation

### Removing Duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. 

In [42]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method **duplicated** returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [43]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, **drop_duplicates** returns a DataFrame where the duplicated array is False

In [44]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates.   
Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column.

In [45]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [46]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


**duplicated** and **drop_duplicates** by default keep the first observed value combination. Passing keep='last' will return the last one.

In [47]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. 

Consider the following hypothetical data collected about various kinds of meat

In [48]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from.   
Let’s write down a mapping of each distinct meat type to the kind of animal.


In [49]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [50]:
meat_to_animal

{'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'}

The **map** method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and others are not.   
Thus, we need to convert each value to lowercase using the **str.lower** Series method:


In [51]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [52]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [53]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work.

In [54]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

Filling in missing data with the fillna method is a special case of more general value replacement.  
As you’ve already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so.   
Let’s consider this Series

In [55]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data.   
To replace these with NA values that pandas understands, we can use replace, producing a new Series (unless you pass inplace=True).

In [56]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the substitute value.

In [57]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes.

In [58]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict.

In [59]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

* Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.   
* You can also modify the axes in-place without creating a new data structure.   

Here’s a simple example:

In [2]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [3]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a map method:

In [4]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

You can assign to index, modifying the DataFrame in-place:


In [5]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [6]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is *rename*:

In [7]:
data.rename(index=str.title, columns=str.upper) #str.title :  returns a string where the first character in every word is upper case

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


**rename** can be used in conjunction with a dict-like object providing new values for a subset of the axis labels.

In [65]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'pikachu'})

,one,two,pikachu,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes. Should you wish to modify a dataset in-place, pass inplace=True:


In [66]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

Continuous data is often discretized or otherwise separated into “bins” for analysis.   
Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets

In [67]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older.  
To do so, you have to use **cut**, a function in pandas.

In [68]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)

In [69]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

An open interval does not include its endpoints, and is indicated with parentheses. For example, (0,1) means greater than 0 and less than 1. This means (0,1) = {x | 0 < x < 1}.

A closed interval is an interval which includes all its limit points, and is denoted with square brackets. For example, [0,1] means greater than or equal to 0 and less than or equal to 1.

The object pandas returns is a special Categorical object.   
The output you see describes the bins computed by pandas.cut.  
You can treat it like an array of strings indicating the bin name; internally it contains a categories array specifying the distinct category names along with a labeling for the ages data in the codes attribute.


In [70]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [71]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [72]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Note that **pd.value_counts**(cats) are the bin counts for the result of **pandas.cut**. 

Consistent with mathematical notation for intervals, a parenthesis means that the side is open, while the square bracket means it is closed (inclusive).  
You can change which side is closed by passing **right=False**:


In [73]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the **labels** option.

In [74]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [75]:
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to **cut** instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.   
Consider the case of some uniformly distributed data chopped into fourths:


In [76]:
data = np.random.rand(20)
data

array([0.4896, 0.3773, 0.8486, 0.9111, 0.3838, 0.3155, 0.5684, 0.1878,
       0.1258, 0.6876, 0.7996, 0.5735, 0.9732, 0.6341, 0.8884, 0.4954,
       0.3516, 0.7142, 0.5039, 0.2256])

In [77]:
data.min()

0.12584154382737023

In [78]:
data.max()

0.9732299815994849

In [79]:
pd.cut(data, 4, precision=2)

[(0.34, 0.55], (0.34, 0.55], (0.76, 0.97], (0.76, 0.97], (0.34, 0.55], ..., (0.34, 0.55], (0.34, 0.55], (0.55, 0.76], (0.34, 0.55], (0.12, 0.34]]
Length: 20
Categories (4, interval[float64, right]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.97]]

In [80]:
data=np.arange(10)
print(data)
pd.cut(data, 2)

[0 1 2 3 4 5 6 7 8 9]


[(-0.009, 4.5], (-0.009, 4.5], (-0.009, 4.5], (-0.009, 4.5], (-0.009, 4.5], (4.5, 9.0], (4.5, 9.0], (4.5, 9.0], (4.5, 9.0], (4.5, 9.0]]
Categories (2, interval[float64, right]): [(-0.009, 4.5] < (4.5, 9.0]]

The **precision=2** option limits the decimal precision to two digits. 

A closely related function, **qcut**, bins the data based on sample quantiles.  
Depending on the distribution of the data, using **cut** will not usually result in each bin having the same number of data points.   
Since **qcut** uses sample quantiles instead, by definition you will obtain roughly equal-size bins:


In [145]:
data = np.random.randn(1000)  # standard normal distribution

In [146]:
cats = pd.qcut(data, 4)  # Cut into quartiles

In [83]:
cats

[(-0.0265, 0.62], (0.62, 3.928], (-0.68, -0.0265], (0.62, 3.928], (-0.0265, 0.62], ..., (-0.68, -0.0265], (-0.68, -0.0265], (-2.9499999999999997, -0.68], (0.62, 3.928], (-0.68, -0.0265]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.68] < (-0.68, -0.0265] < (-0.0265, 0.62] < (0.62, 3.928]]

In [84]:
pd.value_counts(cats)

(-2.9499999999999997, -0.68]    250
(-0.68, -0.0265]                250
(-0.0265, 0.62]                 250
(0.62, 3.928]                   250
dtype: int64

Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive).

In [85]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0265, 1.286], (-0.0265, 1.286], (-1.187, -0.0265], (-0.0265, 1.286], (-0.0265, 1.286], ..., (-1.187, -0.0265], (-1.187, -0.0265], (-2.9499999999999997, -1.187], (-0.0265, 1.286], (-1.187, -0.0265]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0265] < (-0.0265, 1.286] < (1.286, 3.928]]

In [86]:
cats=pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
pd.value_counts(cats)

(-1.187, -0.0265]                400
(-0.0265, 1.286]                 400
(-2.9499999999999997, -1.187]    100
(1.286, 3.928]                   100
dtype: int64

### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations.   
Consider a DataFrame with some normally distributed data:

In [147]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.022434,-0.004741,-0.058187,0.044673
std,1.008762,0.995184,0.991329,0.997007
min,-3.184377,-3.745356,-3.428254,-3.645860
25%,-0.628122,-0.697084,-0.747478,-0.599807
50%,-0.013609,-0.029924,-0.091364,0.047101
75%,0.695298,0.694459,0.618965,0.740562
max,3.525865,2.735527,3.366626,2.653656


In [148]:
data

,0,1,2,3
0,0.884111,-0.608506,-0.072052,0.544066
1,0.323886,-1.683325,0.526860,1.858791
2,-0.548419,-0.279397,-0.021299,-0.287990
3,0.089175,0.522858,0.572796,-1.760372
4,1.128179,1.568606,-0.342277,-0.009813
...,...,...,...,...
995,-0.005261,0.308222,-0.522163,0.175056
996,-1.029444,2.238196,-0.928520,-0.587842
997,1.429361,-2.173409,0.349901,0.863522
998,0.511796,1.761981,-0.681278,0.591952


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value.

In [149]:
col = data[2]
col[np.abs(col) > 3]

270   -3.428254
647    3.366626
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the any method on a boolean DataFrame.

In [150]:
data[(np.abs(data) > 3).any(axis=1)] # axis=1 은 row 중에서 3이상인것이 있으면 그 row는 True

,0,1,2,3
53,-0.025907,-3.399312,-0.974657,-0.685312
72,3.260383,0.963301,1.201206,-1.852001
148,-0.196713,-3.745356,-1.520113,-0.346839
247,-3.056990,1.918403,-0.578828,1.847446
270,0.326045,0.425384,-3.428254,-0.296336
334,-3.184377,1.369891,-1.074833,-0.089937
555,0.208011,-0.150923,-0.362528,-3.548824
647,0.193299,1.397822,3.366626,-2.372214
794,3.525865,0.283070,0.544635,0.462204
814,-0.450721,-0.080332,0.599947,-3.645860


Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3.

In [151]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [152]:
data

,0,1,2,3
0,0.884111,-0.608506,-0.072052,0.544066
1,0.323886,-1.683325,0.526860,1.858791
2,-0.548419,-0.279397,-0.021299,-0.287990
3,0.089175,0.522858,0.572796,-1.760372
4,1.128179,1.568606,-0.342277,-0.009813
...,...,...,...,...
995,-0.005261,0.308222,-0.522163,0.175056
996,-1.029444,2.238196,-0.928520,-0.587842
997,1.429361,-2.173409,0.349901,0.863522
998,0.511796,1.761981,-0.681278,0.591952


In [154]:
data.iloc[53]  # 3 셀 앞의 결과에 나온 숫자 입력

0   -0.025907
1   -3.000000
2   -0.974657
3   -0.685312
Name: 53, dtype: float64

In [94]:
data.iloc[60]  # 3 셀 앞의 결과에 나온 숫자 입력

0    1.951312
1    3.000000
2    0.963301
3    1.201206
Name: 60, dtype: float64

In [95]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.050286,0.025567,-0.001399,-0.051765
std,0.992920,1.004214,0.991414,0.995761
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


The statement **np.sign(data)** produces 1 and –1 values based on whether the values in data are positive or negative:


In [155]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,1.0,-1.0,1.0,1.0
2,-1.0,-1.0,-1.0,-1.0
3,1.0,1.0,1.0,-1.0
4,1.0,1.0,-1.0,-1.0


In [156]:
data.head()

,0,1,2,3
0,0.884111,-0.608506,-0.072052,0.544066
1,0.323886,-1.683325,0.526860,1.858791
2,-0.548419,-0.279397,-0.021299,-0.287990
3,0.089175,0.522858,0.572796,-1.760372
4,1.128179,1.568606,-0.342277,-0.009813


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the numpy.random.permutation function.   
Calling permutation with the length of the axis you want to permute produces an array of integers indicating the new ordering.


In [157]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [159]:
sampler = np.random.permutation(5)
sampler

array([1, 2, 4, 0, 3])

That array can then be used in iloc-based indexing or the equivalent **take** function.

In [160]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [161]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15


To select a random subset without replacement, you can use the **sample** method on Series and DataFrame:

In [162]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19


To generate a sample with replacement (to allow repeat choices), pass **replace=True** to sample.

In [103]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

4    4
1    7
4    4
2   -1
0    5
3    6
1    7
4    4
0    5
4    4
dtype: int64

### Computing Indicator/Dummy Variables

* Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix. 
* If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s.
* pandas has a get_dummies function for doing this, though devising one yourself is not difficult. 

Let’s return to an earlier example DataFrame

In [164]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})

In [165]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [106]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data.   
**get_dummies** has a prefix argument for doing this:


In [107]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [108]:
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [109]:
df_with_dummy = df[['data1']].join(dummies)

In [110]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


If a row in a DataFrame belongs to multiple categories, things are a bit more complicated.   
Let’s look at the MovieLens 1M dataset, which is investigated in more detail in Chapter 14:


In [111]:
mnames = ['movie_id', 'title', 'genres']

In [112]:
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames, encoding="ISO-8859-1")

C:\Users\NN\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [113]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding indicator variables for each genre requires a little bit of wrangling.  
First, we extract the list of unique genres in the dataset:


In [114]:
movies.iloc[0,2].split('|')

['Animation', "Children's", 'Comedy']

In [115]:
movies.genres[0].split('|')

['Animation', "Children's", 'Comedy']

In [116]:
all_genres = []

In [117]:
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [118]:
genres = pd.unique(all_genres)

In [119]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct the indicator DataFrame is to start with a DataFrame of all zeros:


In [120]:
zero_matrix = np.zeros((len(movies), len(genres)))

In [121]:
zero_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [122]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [123]:
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, iterate through each movie and set entries in each row of dummies to 1.  
To do this, we use the **dummies.columns** to compute the column indices for each genre:


In [124]:
gen = movies.genres[0]

In [125]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [126]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

Then, we can use **.iloc** to set values based on these indices:


In [127]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [128]:
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Then, as before, you can combine this with movies.

In [129]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [130]:
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                             1.0
Genre_Children's                            1.0
                               ...             
Genre_War                                   0.0
Genre_Musical                               0.0
Genre_Mystery                               0.0
Genre_Film-Noir                             0.0
Genre_Western                               0.0
Name: 0, Length: 21, dtype: object

In [131]:
movies_windic

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,3949,Requiem for a Dream (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,3950,Tigerland (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,3951,Two Family House (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<img style="float: left;" src="pic/pic_0_2.png">

For much larger data, this method of constructing indicator variables with multiple membership is not especially speedy.   
It would be better to write a lower-level function that writes directly to a NumPy array, and then wrap the result in a DataFrame.

A useful recipe for statistical applications is to combine get_dummies with a discretization function like cut:

In [132]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [133]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [134]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


We will look again at **pandas.get_dummies** later in the book. 

## 7.3  String Manipulation

Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing.   
Most text operations are made simple with the string object’s built-in methods.  
For more complex pattern matching and text manipulations, regular expressions may be needed.  
pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data. 

### String Object Methods

In many string munging and scripting applications, built-in string methods are sufficient.   
As an example, a comma-separated string can be broken into pieces with **split**.

In [135]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

split is often combined with **strip** to trim whitespace (including line breaks).

In [136]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

These substrings could be concatenated together with a two-colon delimiter using addition.

In [137]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the **join** method on the string '::'.

In [138]:
'::'.join(pieces)

'a::b::guido'

Other methods are concerned with locating substrings. Using Python’s in keyword is the best way to detect a substring, though **index** and **find** can also be used.

In [139]:
val

'a,b,  guido'

In [140]:
'guido' in val

True

In [141]:
val.index(',')

1

In [142]:
val.find(',')

1

In [143]:
val.find(':')

-1

Note the difference between find and index is that index raises an exception if the string isn’t found (versus returning –1).

In [144]:
val.index(':')

ValueError: substring not found

 **count** returns the number of occurrences of a particular substring.

In [166]:
val.count(',')

2

In [167]:
val.count(':')

0

**replace** will substitute occurrences of one pattern for another.   
It is commonly used to delete patterns, too, by passing an empty string

In [168]:
val.replace(',', '::')

'a::b::  guido'

In [169]:
val.replace(',', '')

'ab  guido'

See Table 7-3 for a listing of some of Python’s string methods.   
Regular expressions can also be used with many of these operations, as you’ll see.


<img style="float: left;" src="pic/pic_7_3.png" width="600">

## 이번 챕터 끝 (다음 주제는 제대로 살펴보려면 시간부족!!)

### Regular Expressions

  
#### Regulare expression 을 제대로 학습하려면 양이 아주 많다.

* *Regular* expressions provide a flexible way to search or match (often more complex) string patterns in text.   
* A single expression, commonly called a *regex*, is a string formed according to the regular expression language.   
* Python’s built-in re module is responsible for applying regular expressions to strings. 


<img style="float: left;" src="pic/pic_0_2.png">

The art of writing regular expressions could be a chapter of its own and thus is outside the book’s scope.   
There are many excellent tutorials and references available on the internet and in other books.


The **re** module functions fall into three categories: pattern matching, substitution, and splitting.   
Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes.

Suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines).   
The regex describing **one or more whitespace characters** is **\s+**:


In [ ]:
import re

In [ ]:
text = "foo    bar\t baz  \tqux"

In [ ]:
text

In [ ]:
re.split('\s+', text)

When you call re.split('\s+', text), the regular expression is first *compiled*, and then its **split** method is called on the passed text.   
You can compile the regex yourself with **re.compile**, forming a reusable regex object

In [ ]:
regex = re.compile('\s+')
regex.split(text)

If, instead, you wanted to get a list of all patterns matching the regex, you can use the findall method.

In [ ]:
regex.findall(text)

<img style="float: left;" src="pic/pic_0_2.png">

To avoid unwanted escaping with \ in a regular expression, use raw string literals like r'C:\x' instead of the equivalent 'C:\\x'.

Creating a regex object with **re.compile** is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles. 

**match** and **search** are closely related to **findall**.   
While **findall** returns all matches in a string, **search** returns only the first match.   
More rigidly, **match** only matches at the beginning of the string.   
As a less trivial example, let’s consider a block of text and a regular expression capable of identifying most email addresses:


In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

Using **findall** on the text produces a list of the email addresses.

In [ ]:
regex.findall(text)

**search** returns a special match object for the first email address in the text.   
For the preceding regex, the match object can only tell us the start and end position of the pattern in the string.

In [ ]:
m = regex.search(text)

In [ ]:
m

In [ ]:
text[m.start():m.end()]

Do not confuse **re.search()** with **re.match()**.   
Both functions do exactly the same, with the important distinction that **re.search()** will attempt the pattern throughout the string, until it finds a match.   
**re.match()** on the other hand, only attempts the pattern at the very start of the string. 

In [ ]:
print(regex.match(text))

In [ ]:
text="steve@gmail.com"
print(regex.match(text))

Relatedly, *sub* will return a new string with occurrences of the pattern replaced by the a new string

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [ ]:
print(regex.sub('REDACTED', text))

Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix.  
To do this, put parentheses around the parts of the pattern to segment.


In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced by this modified regex returns a tuple of the pattern components with its **groups** method.

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

**findall** returns a list of tuples when the pattern has groups.

In [ ]:
regex.findall(text)

**sub** also has access to groups in each match using special symbols like \1 and \2.   
The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:


In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

There is much more to regular expressions in Python, most of which is outside the book’s scope. Table 7-4 provides a brief summary.

<img style="float: left;" src="pic/pic_7_4.png" width="600">

### Vectorized String Functions in pandas

Cleaning up a messy dataset for analysis often requires a lot of string munging and regularization.   
To complicate matters, a column containing strings will sometimes have missing data.

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

In [ ]:
data.isnull()

You can apply string and regular expression methods can be applied (passing a **lambda** or other function) to each value using **data.map**, but it will fail on the NA (null) values.   
To cope with this, Series has array-oriented methods for string operations that skip NA values.   
These are accessed through Series’s **str** attribute; for example, we could check whether each email address has 'gmail' in it with **str.contains**.


In [ ]:
data.str.contains('gmail')

Regular expressions can be used, too, along with any re options like IGNORECASE.

In [ ]:
pattern

In [ ]:
data.str.findall(pattern, flags=re.IGNORECASE)

There are a couple of ways to do vectorized element retrieval.   
Either use **str.get** or index into the **str** attribute

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

To access elements in the embedded lists, we can pass an index to either of these functions.

In [ ]:
matches.str.get(0)

In [ ]:
type(matches)

In [ ]:
type(matches.str)

In [ ]:
type(matches.astype(str))

In [ ]:
matches

In [ ]:
type(matches.astype(str).str)

In [ ]:
matches.astype(str).str.get(0)

In [ ]:
matches.astype(str).str[0]

In [ ]:
matches.astype(str)

In [ ]:
matches.str[0]

In [ ]:
matches=data.str.findall(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(0)

In [ ]:
matches.str[0]

You can similarly slice strings using this syntax.

In [ ]:
data

In [ ]:
data.str[:5]

<img style="float: left;" src="pic/pic_7_5.png" width="600">

<img style="float: left;" src="pic/pic_7_6.png" width="600">